In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

In [2]:
import scanpy as sc 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches

In [3]:
test_adata = sc.read_h5ad('/ix/djishnu/shared/djishnu_kor11/training_data_2025/mouse_kidney_visiumHD.h5ad')
ref_adata = sc.read_h5ad('/ix/djishnu/shared/djishnu_kor11/training_data_2025/mouse_kidney_13.h5ad')
ref_models_dir = '/ix/djishnu/shared/djishnu_kor11/lasso_runs/mouse_kidney_13'

ref_adata, test_adata

(AnnData object with n_obs × n_vars = 8509 × 3058
     obs: 'ct1', 'ct2', 'ct3', 'ct4', 'cond', 'medulla_cortex', 'domain', 'cell_type', 'cell_type_int'
     uns: 'cell_thresholds', 'cell_type_colors', 'neighbors', 'pca', 'received_ligands', 'received_ligands_tfl', 'umap'
     obsm: 'X_pca', 'X_umap', 'spatial', 'spatial_unscaled'
     varm: 'PCs'
     layers: 'imputed_count', 'normalized_count', 'raw_count'
     obsp: 'connectivities', 'distances',
 AnnData object with n_obs × n_vars = 10000 × 3058
     obs: 'cell_type'
     uns: 'cell_type_colors', 'log1p'
     obsm: 'spatial', 'spatial_unscaled'
     layers: 'normalized_count', 'raw_count')

In [4]:
import sys
sys.path.append('/ix3/djishnu/alw399/SpaceOracle/src')
from spaceoracle.visionary import Visionary
from spaceoracle.oracles import BaseTravLR

BaseTravLR.impute_clusterwise(test_adata, annot='cell_type', layer='normalized_count', layer_added='imputed_count')


2025-07-03 12:49:03.252263: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /ihome/crc/install/gcc-12.2.0/postgresql/15.2/lib:/ihome/crc/install/gcc-12.2.0/imagemagick/6.9.12/lib:/ihome/crc/install/gcc-12.2.0/geos/3.9.4/lib:/ihome/crc/install/gcc-12.2.0/zstd/zstd-1.5.5/lib:/ihome/crc/install/gcc-12.2.0/unixodbc/2.3.11/lib:/ihome/crc/install/sodium/1.0.18/lib:/ihome/crc/install/gcc-12.2.0/r/4.4.0/lib64/R/lib:/ihome/crc/install/gcc-12.2.0/openmpi/4.1.5/lib:/ihome/crc/install/gcc-12.2.0/fftw/3.3.10/lib:/ihome/crc/install/gcc-12.2.0/glpk/5.0/lib:/ihome/crc/install/gcc-12.2.0/proj/4.9.3/lib:/ihome/crc/install/gcc-12.2.0/gdal/2.4.4/lib:/ihome/crc/install/gcc-12.2.0/netcdf/4.8.1/lib:/ihome/crc/install/gcc-12.2.0/hdf5/1.14.0/lib:/ihome/crc/install/gcc-12.2.0/jags/4.3.2/lib:/ihome/crc/install/gcc-12.2.0/gsl/2.7.1/lib:/ihome/c

In [5]:
matching = pd.read_csv('matching_13_HD.csv', index_col=0)

In [6]:
import warnings
warnings.simplefilter("ignore", category=UserWarning)

In [8]:
renata = Visionary(
    ref_adata = ref_adata,
    test_adata = test_adata,
    ref_json_path = f'{ref_models_dir}/run_params.json',
    prematching=matching,
    matching_annot='cell_type'
)

In [9]:
target = 'Mif' 
gex = 0

In [10]:
epithelial = np.where(test_adata.obs.cell_type == 'Epithelial')[0]
len(epithelial)

5437

In [11]:
renata.perturb(
    target, 
    n_propagation=4, 
    save_layer=True,
    cells=epithelial
)

gex_df = renata.adata.to_df(layer=f'{target}_4n_{gex}x')
gex_df.to_parquet(f'/ix/djishnu/shared/djishnu_kor11/genome_screens/mouse_kidney_visiumHD/{target}_4n_{gex}x.parquet')